In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
filename = 'E:/GZ-DESI/data/raw-cats/_desi_pytorch_v5_hpv2_train_all_notest_ml_catalog_x5_advanced.parquet'

In [3]:
df = pd.read_parquet(filename, engine = 'pyarrow', columns = ['id_str', 'merging_none_fraction', 'merging_minor-disturbance_fraction', 'merging_major-disturbance_fraction', 'merging_merger_fraction'])

In [4]:
df

,id_str,merging_none_fraction,merging_minor-disturbance_fraction,merging_major-disturbance_fraction,merging_merger_fraction
0,388972_4019,0.808271,0.129232,0.035381,0.027116
1,388972_4570,0.765248,0.095092,0.063911,0.075749
2,388972_4156,0.853088,0.110454,0.022176,0.014282
3,388972_5110,0.882172,0.082921,0.020319,0.014588
4,388973_5576,0.868599,0.098728,0.022844,0.009828
...,...,...,...,...,...
8689365,444048_423,0.742853,0.135686,0.047769,0.073692
8689366,444048_424,0.796766,0.096348,0.046435,0.060451
8689367,444048_774,0.839374,0.127835,0.020739,0.012052
8689368,444048_663,0.742348,0.161078,0.032188,0.064385


In [5]:
def categorising(none, minor, major, merger):
    if none > minor and none > major and none > merger:
        if none < 0.50:
            return 'uncertain'
        else:
            return 'none'
    elif minor > none and minor > major and minor > merger:
        if minor < 0.50:
            return 'uncertain'
        else:
            return 'minor_disturbance'
    elif major > none and major > minor and major > merger:
        if major < 0.50:
            return 'uncertain'
        else:
            return 'major_disturbance'
    elif merger > none and merger > minor and merger > major:
        if merger < 0.50:
            return 'uncertain'
        else:
            return 'merger'
    
    if none == minor or none == major or none == merger:
        return 'investigate'
    elif minor == major or minor == merger:
        return 'investigate'
    elif major == merger:
        return 'investigate'
    
    return 'Did not work'

In [6]:
df_cat = (
    df
    .assign(category = df.progress_apply(lambda row: categorising(row.merging_none_fraction, row['merging_minor-disturbance_fraction'], row['merging_major-disturbance_fraction'], row.merging_merger_fraction), axis = 1))
)

100%|██████████| 8689370/8689370 [07:36<00:00, 19048.87it/s]


In [7]:
len(df_cat)

8689370

In [8]:
df_cat.category.value_counts()

none                 7744244
uncertain             747987
merger                185278
major_disturbance      11594
minor_disturbance        267
Name: category, dtype: int64

In [9]:
df_cat.category.value_counts() / len(df_cat)

none                 0.891232
uncertain            0.086081
merger               0.021322
major_disturbance    0.001334
minor_disturbance    0.000031
Name: category, dtype: float64

In [11]:
df_non = df_cat.query('category == "none"')

In [10]:
df_int = df_cat.query('category != "none"')

In [12]:
df_def = df_int.query('category != "uncertain"')

In [13]:
df_def.to_parquet('E:\GZ-DESI\data\definitive-predicted-int.parquet')

In [14]:
df_uncertain = df_int.query('category == "uncertain"')

In [15]:
df_uncertain.to_parquet('E:/GZ-DESI/data/uncertain-predicted-int.parquet')

In [16]:
df_non.to_parquet('E:/GZ-DESI/data/non-interacting-predicted-int.parquet')